# indexing the rhyme

In [1]:
import os
import sys
import logging
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)
from core.utils import (
    vn_grammar_handler,
    db_handler,
    init_logger)
init_logger()

/mnt/d/personal_working/bavandangvanba/


In [2]:
import pandas as pd
n_gram_phrases_song = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_songs_data.pkl'))
n_gram_phrases_truyen = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_truyen_data.pkl'))
n_gram_phrases = n_gram_phrases_truyen
for key,count in n_gram_phrases_song.items():
    if key not in n_gram_phrases:
        n_gram_phrases[key] = count*10
    else:
        n_gram_phrases[key] += count*10


In [3]:
len(n_gram_phrases)

1514931

In [10]:
wrong_data = []
van_indexing = {} 
for idx,(word, count) in enumerate(n_gram_phrases.items()):
    cleaned_text = vn_grammar_handler.bo_dau_va_chuyen_van(word)
    matches_01 = vn_grammar_handler.lay_van_cua_tu_hoac_doan(cleaned_text)
    matches_02 = vn_grammar_handler.clean_phu_am(cleaned_text)
    if matches_01!=matches_02:
        wrong_data.append(['something wrong', word, matches_01, matches_02])
    if matches_01 not in van_indexing:
        van_indexing[matches_01] = []
    van_indexing[matches_01].append((word,count))
for van in van_indexing:
    van_indexing[van].sort(key=lambda x:x[1],reverse=True)
del van_indexing[""]


In [12]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS van_indexing
                  (id TEXT PRIMARY KEY, words TEXT)''')

In [13]:
for idx,(van, text) in enumerate(van_indexing.items()):
    text_list = "\n".join(x[0] for x in text)
   # Add data to the table
    data = ('1', 'Hello, world!')
    cursor.execute('INSERT INTO van_indexing (id, words) VALUES (?, ?)', (van,text_list))
    if idx % 10000 == 0:
        logging.info(f'  - processed {idx} van')

2023-11-11 05:04:09 INFO       - processed 0 van
2023-11-11 05:04:09 INFO       - processed 10000 van
2023-11-11 05:04:10 INFO       - processed 20000 van
2023-11-11 05:04:10 INFO       - processed 30000 van
2023-11-11 05:04:10 INFO       - processed 40000 van
2023-11-11 05:04:10 INFO       - processed 50000 van
2023-11-11 05:04:10 INFO       - processed 60000 van
2023-11-11 05:04:10 INFO       - processed 70000 van
2023-11-11 05:04:10 INFO       - processed 80000 van
2023-11-11 05:04:10 INFO       - processed 90000 van
2023-11-11 05:04:11 INFO       - processed 100000 van
2023-11-11 05:04:11 INFO       - processed 110000 van
2023-11-11 05:04:12 INFO       - processed 120000 van
2023-11-11 05:04:12 INFO       - processed 130000 van
2023-11-11 05:04:13 INFO       - processed 140000 van
2023-11-11 05:04:14 INFO       - processed 150000 van
2023-11-11 05:04:15 INFO       - processed 160000 van
2023-11-11 05:04:16 INFO       - processed 170000 van
2023-11-11 05:04:17 INFO       - processed

In [14]:
conn.commit()

# Close the connection
conn.close()

In [15]:
import subprocess

source_file_path = "/mnt/d/personal_working/bavandangvanba/app_metadata/example.db"
destination_file_path = "giangtran@ec2-52-74-160-204.ap-southeast-1.compute.amazonaws.com:/home/giangtran/working/bavandangvanba/app_metadata/example.db"
key_file_path = "/home/giangtran/.ssh/giang_key.pem"

subprocess.run(["scp", "-i", key_file_path, source_file_path, destination_file_path])

CompletedProcess(args=['scp', '-i', '/home/giangtran/.ssh/giang_key.pem', '/mnt/d/personal_working/bavandangvanba/app_metadata/example.db', 'giangtran@ec2-52-74-160-204.ap-southeast-1.compute.amazonaws.com:/home/giangtran/working/bavandangvanba/app_metadata/example.db'], returncode=0)